In [1]:
import numpy as np
import pandas as pd
import fastFM as fm
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

In [2]:
import util

In [3]:
df_train = pd.read_csv('../data/ml-100k/ua.base', header=None, names=['uid', 'sid', 'score', 'ts'], sep='\t')

In [4]:
df_test = pd.read_csv('../data/ml-100k/ua.test', header=None, names=['uid', 'sid', 'score', 'ts'], sep='\t')

In [5]:
df = df_train

In [6]:
df.head()

,uid,sid,score,ts
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [7]:
!wc -l ../data/ml-100k/ua.base

   90570 ../data/ml-100k/ua.base


In [8]:
!wc -l ../data/ml-100k/ua.test

    9430 ../data/ml-100k/ua.test


In [9]:
import imp
util = imp.reload(util)
X_base, Y_base = util.load_dataset_mf('../data/ml-100k/ua.base', user_cnt=943, item_cnt=1682, entry_cnt=90570)
X_test, Y_test = util.load_dataset_mf('../data/ml-100k/ua.test', user_cnt=943, item_cnt=1682, entry_cnt=9430)

In [10]:
### classification

In [11]:
y_base = np.ones_like(Y_base)
y_test = np.ones_like(Y_test)
y_base[Y_base < np.mean(Y_base)] = 0
y_base[Y_base >= np.mean(Y_base)] = 1
y_test[Y_test < np.mean(Y_test)] = 0
y_test[Y_test >= np.mean(Y_base)] = 1
X_train, X_dev, y_train, y_dev = train_test_split(X_base.tocsr(), y_base)

In [12]:
# check balance
print('threshold', np.mean(Y_base))
print(np.count_nonzero(y_train)/len(y_train))
print(np.count_nonzero(y_dev)/len(y_dev))
print(np.count_nonzero(y_test)/len(y_test))
y_train[y_train == 0] = -1
y_dev[y_dev == 0] = -1
y_test[y_test == 0] = -1

threshold 3.5238268
0.550708849205765
0.5519586627213708
0.5799575821845175


In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(67927, 2625)
(67927,)
(9430, 2625)
(9430,)


In [17]:
from fastFM import sgd
fm = sgd.FMClassification(n_iter=100000, init_stdev=0.1, l2_reg_w=0.1,
                          l2_reg_V=0.1, rank=5, step_size=0.01)
fm.fit(X_train, y_train)
util.evaluate_classification(fm, X_train, y_train)
util.evaluate_classification(fm, X_dev, y_dev)
util.evaluate_classification(fm, X_test, y_test)

acc: 0.6712205750290753
auc: 0.7545273257882432
acc: 0.6625447158062094
auc: 0.7378575466111048
acc: 0.6498409331919406
auc: 0.7017339336460643


In [ ]:
### regression

In [37]:
X_train, X_dev, y_train, y_dev = train_test_split(X_base, Y_base)
y_test = Y_test

In [27]:
from fastFM import als
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=5, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)

mse: 0.5177298742770168
mse: 1.0660045369488942
mse: 1.1919620954550223


In [30]:
ytr = util.evaluate_regression(fm, X_train, y_train)
yde = util.evaluate_regression(fm, X_dev, y_dev)
yte = util.evaluate_regression(fm, X_test, Y_test)

mse: 0.5177298742770168
mse: 1.0660045369488942
mse: 1.1919620954550223


In [29]:
util = imp.reload(util)

In [38]:
print(list(zip(ytr[:5], y_train[:5])))
print(list(zip(yde[:5], y_dev[:5])))
print(list(zip(yte[:5], y_test[:5])))

[(3.5226846100657188, 4.0), (4.002202633311479, 4.0), (3.6670146626794855, 2.0), (4.193538725097316, 3.0), (2.141404436289637, 1.0)]
[(4.501392303233024, 4.0), (3.0771562108111095, 2.0), (4.08150303279746, 2.0), (3.7367534676111873, 3.0), (3.6711764251274293, 4.0)]
[(4.365043829341479, 4.0), (3.098901866183153, 4.0), (3.777481714255993, 4.0), (3.189806025955311, 3.0), (2.220694634835228, 2.0)]
